<a href="https://colab.research.google.com/github/NNoorFatima/mBart-roman-urdu/blob/main/roman_urdu_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install evaluate if not already installed
!pip install evaluate --quiet
!pip install sacrebleu --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
# !pip install -q accelerate


In [ ]:
# !pip install -q bitsandbytes


In [ ]:
# !pip install -q -i https://test.pypi.org/simple/ bitsandbytes


In [2]:
!pip uninstall -y transformers peft accelerate
!pip install transformers==4.35.2 peft==0.6.2 accelerate==0.23.0


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
ERROR: pip's dependency resolver does not currently take into account all

In [3]:
import random
import os
import pandas as pd
from datasets import Dataset
# from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    TrainerCallback
)
import evaluate
import torch


/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [4]:
# Load CSV
df = pd.read_csv("/content/ERUPD_NMT.csv")  # update path if needed
df = df[["English", "Roman Urdu"]]      # select relevant columns

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

print("First 5rows:\n", dataset[:5])


First 5rows:
 {'English': ['In the heart of the bustling city, lived a young woman named Aisha', 'She had always dreamed of exploring the world beyond the familiar streets and lively markets of her hometown', 'One day, as she perused a travel magazine, Aisha stumbled upon an article about an enchanting village nestled in the mountains', "The vivid descriptions of lush landscapes and the warmth of the villagers' hospitality ignited a spark in her adventurous spirit", 'Determined to turn her dreams into reality, Aisha decided to embark on a solo journey'], 'Roman Urdu': ['Shehar ki dil mein rehti thi ek nojawan aurat jiska naam Aisha tha', 'Usne hamesha khwab dekha tha ke apne sheher ke maqami galiyon aur zinda bazaron ke bahar duniya ko tajjub bhari nigahon se dekhe', 'Ek din, jab usne ek safar nama parha, Aisha ne ek pahaaroon mein basa hua jadoo bhara gaon ke baare mein ek maqala parha', 'Yehan ke hari bhari manazir aur gaon walon ki mehman nawazi ki zindah tasveerain uske jazbati ruh

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [16]:
# Add new language token for Roman Urdu
new_lang_token = "<roman_ur>"
tokenizer.add_tokens([new_lang_token])
tokenizer.tgt_lang = new_lang_token

# Resize model embeddings to include new token
model.resize_token_embeddings(len(tokenizer))

Embedding(250055, 1024, padding_idx=1)

In [17]:
def preprocess(batch):
    sources = ["<en_XX> " + str(t) for t in batch["English"]]
    targets = [f"{new_lang_token} " + str(t) for t in batch["Roman Urdu"]]

    model_inputs = tokenizer(
        sources,
        max_length=128,
        truncation=True,
        padding="max_length",
        text_target=targets,
        return_tensors="pt"
    )

    return {k: v.tolist() for k, v in model_inputs.items()}

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/75146 [00:00<?, ? examples/s]

In [18]:
eval_dataset = tokenized_dataset.select(range(1000))
train_dataset = tokenized_dataset.select(range(1000, len(tokenized_dataset)))


In [19]:
bleu = evaluate.load("sacrebleu")

def compute_metrics(pred):
    preds = tokenizer.batch_decode(pred.predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(pred.label_ids, skip_special_tokens=True)

    labels = [[l] for l in labels]   # sacrebleu format

    result = bleu.compute(predictions=preds, references=labels)
    return {"bleu": result["score"]}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
class TranslationProgressCallback(TrainerCallback):

    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"\n Epoch {int(state.epoch)} completed")
        print(f"➡ Training Loss: {state.log_history[-1]['loss'] if 'loss' in state.log_history[-1] else 'N/A'}")

        # Every 5 epochs → Show example translation
        if int(state.epoch) % 5 == 0:
            example_text = "Where is my order?"
            inputs = tokenizer("<en_XX> " + example_text, return_tensors="pt").to(model.device)
            output_tokens = model.generate(**inputs, max_new_tokens=60)
            translation = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

            print(f" Test English Input: {example_text}")
            print(f" Model Output (Roman Urdu): {translation}")
            print("----------------------------------------------------------")


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# training_args = TrainingArguments(
#     output_dir="./mbart-roman-ur-finetune",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     logging_strategy="steps",
#     logging_steps=100,
#     save_total_limit=5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     learning_rate=5e-5,
#     num_train_epochs=10,        # need 20 so 5-epoch checkpoints make sense
#     report_to="none"
# )
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./mbart-roman-ur-finetune",

    # --- MEMORY SAVERS ---
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,   # effective batch size = 16
    fp16=True,
    gradient_checkpointing=True,

    # --- TRAINING SCHEDULE ---
    num_train_epochs=10,
    learning_rate=5e-5,

    # --- LOGGING & SAVING ---
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=5,
    report_to="none",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[TranslationProgressCallback()]
)


/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:437: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()


You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing`. Setting `use_cache=False`...



 Epoch 0 completed
➡ Training Loss: 0.261
 Test English Input: Where is my order?
 Model Output (Roman Urdu): <roman_ur><roman_ur> mera order?
----------------------------------------------------------


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.03 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.93 GiB is free. Process 6927 has 12.81 GiB memory in use. Of the allocated memory 11.16 GiB is allocated by PyTorch, and 1.52 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.save_pretrained("epoch1_saved")
tokenizer.save_pretrained("epoch1_saved")


('epoch1_saved/tokenizer_config.json',
 'epoch1_saved/special_tokens_map.json',
 'epoch1_saved/sentencepiece.bpe.model',
 'epoch1_saved/added_tokens.json',
 'epoch1_saved/tokenizer.json')

In [ ]:
!zip -r epoch1_saved.zip epoch1_saved


  adding: epoch1_saved/ (stored 0%)
  adding: epoch1_saved/generation_config.json (deflated 42%)
  adding: epoch1_saved/added_tokens.json (stored 0%)
  adding: epoch1_saved/tokenizer_config.json (deflated 92%)
  adding: epoch1_saved/config.json (deflated 59%)
  adding: epoch1_saved/special_tokens_map.json (deflated 61%)
  adding: epoch1_saved/tokenizer.json (deflated 76%)
  adding: epoch1_saved/sentencepiece.bpe.model (deflated 49%)
  adding: epoch1_saved/model.safetensors (deflated 7%)


In [ ]:
from google.colab import files
files.download("epoch1_saved.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp -r epoch1_saved /content/drive/MyDrive/


In [ ]:
!cp epoch1_saved.zip /content/drive/MyDrive/


In [ ]:
model = MBartForConditionalGeneration.from_pretrained("epoch1_saved")
tokenizer = MBart50TokenizerFast.from_pretrained("epoch1_saved")


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# 1️⃣ Load the model & tokenizer
model = MBartForConditionalGeneration.from_pretrained("mbart-roman-ur-finetune-epoch-0")
tokenizer = MBart50TokenizerFast.from_pretrained("mbart-roman-ur-finetune-epoch-0")

# 2️⃣ Recreate data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 3️⃣ Recreate training arguments
training_args = TrainingArguments(
    output_dir="./mbart-roman-ur-finetune",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    fp16=True,
    gradient_checkpointing=True,
    num_train_epochs=10,
    learning_rate=5e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=5,
    report_to="none",
)

# 4️⃣ Recreate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[TranslationProgressCallback()]
)

# 5️⃣ Resume training from the saved model
trainer.train()


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.save_model(f"mbart-roman-ur-finetune-epoch-{trainer.state.epoch}")
tokenizer.save_pretrained(f"mbart-roman-ur-finetune-epoch-{trainer.state.epoch}")


('mbart-roman-ur-finetune-epoch-0.005394761686402503/tokenizer_config.json',
 'mbart-roman-ur-finetune-epoch-0.005394761686402503/special_tokens_map.json',
 'mbart-roman-ur-finetune-epoch-0.005394761686402503/sentencepiece.bpe.model',
 'mbart-roman-ur-finetune-epoch-0.005394761686402503/added_tokens.json',
 'mbart-roman-ur-finetune-epoch-0.005394761686402503/tokenizer.json')

In [ ]:
!ls -lh mbart-roman-ur-finetune-epoch-0.005394761686402503


total 2.3G
-rw-r--r-- 1 root root   27 Nov 21 18:33 added_tokens.json
-rw-r--r-- 1 root root 1.4K Nov 21 18:30 config.json
-rw-r--r-- 1 root root  256 Nov 21 18:30 generation_config.json
-rw-r--r-- 1 root root 2.3G Nov 21 18:32 model.safetensors
-rw-r--r-- 1 root root 4.9M Nov 21 18:33 sentencepiece.bpe.model
-rw-r--r-- 1 root root 1.7K Nov 21 18:33 special_tokens_map.json
-rw-r--r-- 1 root root  11K Nov 21 18:33 tokenizer_config.json
-rw-r--r-- 1 root root  17M Nov 21 18:33 tokenizer.json
-rw-r--r-- 1 root root 4.9K Nov 21 18:33 training_args.bin


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("mbart-roman-ur-finetune-epoch-0.005394761686402503")
tokenizer = MBart50TokenizerFast.from_pretrained("mbart-roman-ur-finetune-epoch-0.005394761686402503")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: mbart-roman-ur-finetune-epoch-0.005394761686402503 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.eval()


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250055, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250055, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNor

In [ ]:
english_texts = [
    "Where is my order?",
    "I want to book a flight to Karachi.",
    "The weather is very nice today.",
]

# MBART50 requires specifying the source language
src_lang = "en_XX"
tgt_lang = "roman_ur"   # assuming Roman Urdu is mapped to this (or whatever you used)


In [ ]:
inputs = tokenizer(english_texts, return_tensors="pt", padding=True)
inputs = {k: v.to(model.device) for k, v in inputs.items()}  # move to GPU if available


In [ ]:
# Example English text
example_text = "I want to book a flight to Karachi."

# Tokenize input (add language code)
inputs = tokenizer("<en_XX> " + example_text, return_tensors="pt").to(model.device)

# Generate translation
output_tokens = model.generate(**inputs, max_new_tokens=60)

# Decode the output
translation = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print("English:", example_text)
print("Roman Urdu:", translation)


NameError: name 'tokenizer' is not defined

loading model

In [ ]:
from google.colab import drive
import zipfile
import os
from transformers import MBart50TokenizerFast, MBartForConditionalGeneration

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define paths
zip_path = '/content/drive/MyDrive/epoch1_saved.zip'  # <-- your zip path
extract_dir = '/content/epoch1_saved'

# 3. Make sure the extraction directory exists
os.makedirs(extract_dir, exist_ok=True)

# 4. Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 5. Automatically find the folder containing tokenizer/model files
subfolders = [f.path for f in os.scandir(extract_dir) if f.is_dir()]
if len(subfolders) == 1:
    model_dir = subfolders[0]  # the actual folder with files
else:
    model_dir = extract_dir  # assume files are directly inside

print("Loading from folder:", model_dir)
print("Files inside:", os.listdir(model_dir))

# 6. Load tokenizer and model
tokenizer = MBart50TokenizerFast.from_pretrained(model_dir)
# model = MBartForConditionalGeneration.from_pretrained(model_dir)

model = MBartForConditionalGeneration.from_pretrained(
    model_dir,
    # load_in_8bit=True,   # <--- important
    device_map="auto"    # <--- auto places layers on GPU/CPU as needed
)
# 7. Test inference
example_text = "Where is my order?"
inputs = tokenizer("<en_XX> " + example_text, return_tensors="pt").to(model.device)

# Specify target language
tgt_lang = "ur_PK"
forced_bos_token_id = tokenizer.lang_code_to_id[tgt_lang]

output_tokens = model.generate(
    **inputs,
    max_new_tokens=60,
    forced_bos_token_id=forced_bos_token_id
)

translation = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print("English:", example_text)
print("Roman Urdu:", translation)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading from folder: /content/epoch1_saved/epoch1_saved
Files inside: ['added_tokens.json', 'sentencepiece.bpe.model', 'tokenizer_config.json', 'model.safetensors', 'generation_config.json', 'special_tokens_map.json', 'config.json', 'tokenizer.json']
English: Where is my order?
Roman Urdu: meri order hai?


In [ ]:
training_args = TrainingArguments(
    output_dir="./mbart-roman-ur-finetune",

    # --- MEMORY SAVERS ---
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    fp16=False,
    bf16=False,

    # --- TRAINING SCHEDULE ---
    num_train_epochs=5,
    learning_rate=5e-5,

    # --- LOGGING & SAVING ---
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    save_total_limit=3,
    report_to="none"
)


In [ ]:
import transformers
print(transformers.__version__)


4.57.1


In [ ]:

from transformers import Trainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[TranslationProgressCallback()]
)


/tmp/ipython-input-2566381184.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
import os
import re

# Find any previous checkpoints
checkpoint_paths = [f for f in os.listdir(training_args.output_dir) if re.match(r"checkpoint-\d+", f)]
if checkpoint_paths:
    latest_checkpoint = max(checkpoint_paths, key=lambda x: int(x.split("-")[1]))
    print(f"Resuming from checkpoint: {latest_checkpoint}")
    trainer.train(resume_from_checkpoint=os.path.join(training_args.output_dir, latest_checkpoint))
else:
    print("executing else")
    trainer.train()  # start from manually loaded model (epoch 1)


executing else


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
trainer.save_model("/content/epoch1_partial_checkpoint")


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
import shutil

# Path to checkpoint
checkpoint_path = "/content/epoch1_partial_checkpoint"

# Path to zip file
zip_path = "/content/epoch1_partial_checkpoint.zip"

# Zip the folder
shutil.make_archive(base_name=checkpoint_path, format='zip', root_dir=checkpoint_path)
print("Checkpoint zipped successfully:", zip_path)


Checkpoint zipped successfully: /content/epoch1_partial_checkpoint.zip


In [ ]:
import shutil

# Destination in your Drive
drive_dest = "/content/drive/MyDrive/epoch1_partial_checkpoint.zip"

# Copy zip to Drive
shutil.copy(zip_path, drive_dest)
print("Checkpoint uploaded to Drive:", drive_dest)


Checkpoint uploaded to Drive: /content/drive/MyDrive/epoch1_partial_checkpoint.zip


In [14]:
from transformers import MBart50TokenizerFast, MBartForConditionalGeneration, MBartConfig
import torch

# Paths
tokenizer_dir = "/content/epoch1_saved/epoch1_saved"         # tokenizer folder
checkpoint_dir = "/content/epoch1_partial_checkpoint"        # partial model checkpoint

# Load tokenizer from epoch1_saved
tokenizer = MBart50TokenizerFast.from_pretrained(tokenizer_dir)

# Load config from tokenizer folder (this ensures all generation parameters are set)
config = MBartConfig.from_pretrained(tokenizer_dir)

# Load model from checkpoint, but with config from tokenizer folder
model = MBartForConditionalGeneration.from_pretrained(
    checkpoint_dir,
    config=config,
    device_map={"": "cpu"}  # or "auto" for GPU
)

# Example inference
example_text = "Where is my order?"
inputs = tokenizer("<en_XX> " + example_text, return_tensors="pt").to(model.device)

tgt_lang = "ur_PK"
forced_bos_token_id = tokenizer.lang_code_to_id[tgt_lang]

output_tokens = model.generate(
    **inputs,
    max_new_tokens=60,
    forced_bos_token_id=forced_bos_token_id
)

translation = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print("English:", example_text)
print("Roman Urdu:", translation)


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


English: Where is my order?
Roman Urdu: mera tarteeb kahan hai?


In [15]:
# Example English text
example_text = "I want to book a flight to Karachi."

# Tokenize input (add language code)
inputs = tokenizer("<en_XX> " + example_text, return_tensors="pt").to(model.device)

# Generate translation
output_tokens = model.generate(**inputs, max_new_tokens=60)

# Decode the output
translation = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print("English:", example_text)
print("Roman Urdu:", translation)


English: I want to book a flight to Karachi.
Roman Urdu: <roman_ur><roman_ur> Karachi ka flight book karna hai.


In [ ]:
import os
import zipfile
from google.colab import drive
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, TrainerCallback

# --- Mount Drive (if not already) ---
drive.mount('/content/drive')

# --- Directories ---
output_dir = "./mbart-roman-ur-finetune"                     # where trainer will save
checkpoint_dir = "/content/epoch1_partial_checkpoint_extracted"        # folder or zip with checkpoint
drive_backup_dir = "/content/drive/MyDrive/mbart_checkpoints"  # backup location on Drive
os.makedirs(drive_backup_dir, exist_ok=True)

# # --- Extract checkpoint zip if needed ---
# if checkpoint_dir.endswith(".zip"):
#     extract_dir = "/content/epoch1_partial_checkpoint_extracted"
#     with zipfile.ZipFile(checkpoint_dir, 'r') as zip_ref:
#         zip_ref.extractall(extract_dir)
#     checkpoint_dir = extract_dir
#     print(f"Checkpoint extracted to {extract_dir}")

# --- Custom Callback to zip and backup checkpoints to Drive ---
class DriveBackupCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        checkpoint_path = control.last_model_checkpoint
        if checkpoint_path:
            zip_path = checkpoint_path + ".zip"
            # Zip checkpoint folder
            with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for root, dirs, files in os.walk(checkpoint_path):
                    for file in files:
                        full_path = os.path.join(root, file)
                        zipf.write(full_path, os.path.relpath(full_path, checkpoint_path))
            # Move to Drive
            os.rename(zip_path, os.path.join(drive_backup_dir, os.path.basename(zip_path)))
            print(f"Checkpoint zipped and backed up to Drive: {os.path.join(drive_backup_dir, os.path.basename(zip_path))}")

# --- TrainingArguments ---
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    fp16=True,
    num_train_epochs=10,
    learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=5,
    report_to="none",
)

# --- Data collator ---
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# --- Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[TranslationProgressCallback(), DriveBackupCallback()],
)

# --- Resume training from checkpoint ---
trainer.train(resume_from_checkpoint=checkpoint_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:437: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
You are resuming training from a checkpoint trained with 4.57.1 of Transformers but your current version is 4.35.2. This is not recommended and could yield to errors or unwanted behaviors.
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in Py

Step,Training Loss,Validation Loss


In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()


6247